# GRPA Coding

Goverment Perfromance and Results Act (GPRA) coding for intake data of recovery specialist for the Institure for Prevention and Recovery (IFPR) at RWJ Baranabas

In [80]:
#importing pandas
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [81]:
# importing data and parsing datetime columns
df = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA_Coding_File.csv", parse_dates = ['ADMIT_DT','FORM_DTTM',
                                                                                                'FORM_UPDATE_DTTM'])

#subsetting to include Peer Recovery Program episodes
df = df[df['EPISODE'] =='IFPR - STAR Program'].sort_values(by = ['FORM_DTTM','MRN']) # <- sorting df

In [82]:
#Renaming column to create ClientID
df.rename(columns = {'CSN':'ClientID'}, inplace = True)
df['ClientID'] = df['ClientID'].str.split(',').str[0]

In [83]:
# function to return all values greater than 30 as 30 to be utilized for days columns
def thirty(x):
    if x > 30:
        return 30 #-- returns value 30 if number greater than 30
    else:
        pass #-- will return same value if number not greater than 30

In [84]:
#Creating function for interview type
def interview_type(row):
    if row['INTERVIEW_TYPE'] == 'Intake':
        return 1
    elif row['INTERVIEW_TYPE'] == '6-month follow-up':
        return 2
    elif row['INTERVIEW_TYPE'] == '3-month follow-up':
        return 4
    elif row['INTERVIEW_TYPE'] == 'Discharge':
        return 5
    
#Creating new column via applying interview_type_code function to values in interview type column
df['InterviewType'] = df.apply(interview_type, axis = 1)

## Creating GPRA 6

In [85]:
# Creating function to create column FLWPClientContacted
def FLWP_ClientContacted (row):
    if row['CLIENT_ABLE_TO_BE_CONTACTED_FOR_FOLLOW_UP'] == 'Yes':
        return 1
    elif row['CLIENT_ABLE_TO_BE_CONTACTED_FOR_FOLLOW_UP'] == 'No':
        return 0
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 2:
        return FLWP_ClientContacted (row)
    else:
        pass
    
#Creating column FLWPClientContacted via applying the function FLWP_ClientContacted (x)
df['FLWPClientContacted'] = df.apply(interview_type, axis = 1)

In [86]:
#Creating function to create column ReceivingServices
def Receiving_Services (row):
    if row['CLIENT_STILL_RECEIVING_PROGRAM_SERVICES'] == 'Yes':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 2:
        return Receiving_Services (row)
    else:
        pass    
#Creating column ReceivingServices via applying the function Receiving_Services (x)
df['ReceivingServices'] = df.apply(interview_type, axis = 1)

In [87]:
#Creating dictionary to rename columns
dictionary10 = {'FOLLOW_UP_STATUS_OF_CLIENT':'FLWPStatus','OTHER_FOLLOW_UP_STATUS':'FLWPStatusSpec',
                'CLIENT_DISCHARGE_DATE':'DischargeDate','CLIENT_DISCHARGE_STATUS':'DischargeStatusCompl',
                'REASON_FOR_TERMINATION':'DischargeStatusTermReason',
                'OTHER_REASON_FOR_TERMINATION':'OtherDischargeStatTermRsnSpec',
                'HIV_TEST_ORDERED_BY_PROGRAM':'jHIVTest','CLIENT_REFERRERD_FOR_HIV_TEST':'jHIVTestResult'}

#Renaming columns
df.rename(columns = dictionary10,inplace = True)

#Creating variables to replace null values
null_col10 = ['jHIVTest','jHIVTestResult']

#Replacing null values with -9
df[null_col10] = df[null_col10].fillna(-9)

# Data Cleaning
df['FLWPStatus'] = df['FLWPStatus'].str[:2]
# df['FLWPStatusSpec'] = df['FLWPStatusSpec'].str.title()

df['DischargeStatusCompl'] = df['DischargeStatusCompl'].str[1:2]
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['DischargeStatusCompl']
    else:
        pass 
df['DischargeStatusCompl'] = df.apply(interview_type, axis = 1)


df['DischargeStatusTermReason'] = df['DischargeStatusTermReason'].str[1:2]
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['DischargeStatusTermReason']
    else:
        pass 
df['DischargeStatusTermReason'] = df.apply(interview_type, axis = 1)
# df['OtherDischargeStatTermRsnSpec'] = df['OtherDischargeStatTermRsnSpec'].str.title()
df['jHIVTest'] = df['jHIVTest'].replace('No',0)
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['jHIVTest']
    else:
        pass 
df['jHIVTest'] = df.apply(interview_type, axis = 1)

df['jHIVTestResult'] = df['jHIVTestResult'].replace('Yes',1)
df['jHIVTestResult'] = df['jHIVTestResult'].replace('No',0)
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['jHIVTestResult']
    else:
        pass 
df['jHIVTestResult'] = df.apply(interview_type, axis = 1)

#Converting df['DischargeDate'] to datetime data type
df['DischargeDate'] = pd.to_datetime(df['DischargeDate'])

#Converting to the mm/dd/yyyy format
df['DischargeDate'] = df['DischargeDate'].dt.strftime('%m/%d/%Y')
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['DischargeDate']
    else:
        pass 
    
df['DischargeDate'] = df.apply(interview_type, axis = 1)

In [88]:
def FLWPStatus (row):
    if row['FLWPClientContacted'] == 1:
        return row['FLWPStatus']
    elif row['FLWPClientContacted'] == 0:
        return 32
    
def interview_type(row):
    if row['InterviewType'] == 2:
        return FLWPStatus (row)
    else:
        pass 
df['FLWPStatus'] = df.apply(interview_type, axis = 1)

In [89]:
def FLWPStatusSpec (row):
    if row['FLWPStatus'] == 32:
        return 'None'
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 2:
        return FLWPStatusSpec (row)
    else:
        pass 
    
df['FLWPStatusSpec'] = df.apply(interview_type, axis = 1)

In [90]:
#Replacing null values with -9
df['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'] = df['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'].fillna(-9)

#Creating function to create column ODPreventionToolsProvided
def ODPrevention_ToolsProvided (row):
    if row['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'] == 'Neither':
        return 4
    elif row['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'] == 'Naloxone':
        return 1
    elif row['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'] == -9:
        return -9
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return ODPrevention_ToolsProvided (row)
    else:
        pass 
    
#Creating column ODPreventionToolsProvided via applying the function ODPrevention_ToolsProvided (x)
df['ODPreventionToolsProvided'] = df.apply(interview_type, axis = 1 )

In [91]:
#Replacing null values with -9
df['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] = df['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'].fillna(-9)

#Creating function to create column VaccinatedStatusCOVID19
def Vaccinated_StatusCOVID19 (row):
    if row['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] == 'Yes':
        return 1
    elif row['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] == 'Refused to answer':
        return -7
    elif row['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] == 'No, client refused vaccination':
        return 4
    elif row['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] == 'No, partially vaccinated with no plan to receive the subsequent vaccination':
        return 3
    elif row['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] == -9:
        return -9
    
def interview_type(row):
    if row['InterviewType'] == 5:
        return Vaccinated_StatusCOVID19 (row)
    else:
        pass 
    
    
#Creating column VaccinatedStatusCOVID19 via applying the function Vaccinated_StatusCOVID19 (x)
df['VaccinatedStatusCOVID19'] = df.apply(interview_type, axis = 1)

In [92]:
#Creating column SvcInpatient
df['SvcInpatient'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcInpatient']
    else:
        pass 
df['SvcInpatient'] = df.apply(interview_type, axis = 1)
    

#Creating column SvcOutpatient
df['SvcOutpatient'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOutpatient']
    else:
        pass 
df['SvcOutpatient'] = df.apply(interview_type, axis = 1)

In [93]:
#Creating columns in which no services were provided

# filtering based on interview type
df['SvcMethadone'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcMethadone']
    else:
        pass 
df['SvcMethadone'] = df.apply(interview_type, axis = 1)


# filtering based on interview type
df['SvcBuprenorphine'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcBuprenorphine']
    else:
        pass 
df['SvcBuprenorphine'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcNaltrexoneShortActing'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcNaltrexoneShortActing']
    else:
        pass 
df['SvcNaltrexoneShortActing'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcNaltrexoneLongActing'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcNaltrexoneLongActing']
    else:
        pass 
df['SvcNaltrexoneLongActing'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcDisulfiram'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcDisulfiram']
    else:
        pass 
df['SvcDisulfiram'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcAcamprosate'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcAcamprosate'] 
    else:
        pass 
df['SvcAcamprosate'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcNicotineReplacement'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcNicotineReplacement'] 
    else:
        pass 
df['SvcNicotineReplacement'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcBupropion'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcBupropion']
    else:
        pass 
df['SvcBupropion'] = df.apply(interview_type, axis = 1)


# filtering based on interview type
df['SvcVarenicline'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcVarenicline']
    else:
        pass 
df['SvcVarenicline'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcResidentialRehab'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcResidentialRehab'] 
    else:
        pass 
df['SvcResidentialRehab'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcHospitalInpatient'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcHospitalInpatient']
    else:
        pass 
df['SvcHospitalInpatient'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcFreeStandingRes'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcFreeStandingRes']
    else:
        pass 
df['SvcFreeStandingRes'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcAmbulatoryDetox'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcAmbulatoryDetox']
    else:
        pass 
df['SvcAmbulatoryDetox'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcAfterCare'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcAfterCare']
    else:
        pass 
df['SvcAfterCare'] = df.apply(interview_type, axis = 1)


In [94]:
#Creating columns with blank values
df['SvcOtherModalities'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOtherModalities']
    else:
        pass
df['SvcOtherModalities'] = df.apply(interview_type, axis = 1)

df['SvcOtherModalitesSpec'] = " "

In [95]:
#Creating column SvcBriefTreatment
df['SvcBriefTreatment'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcBriefTreatment']
    else:
        pass
df['SvcBriefTreatment'] = df.apply(interview_type, axis = 1)


In [96]:
#Creating dictionary to rename columns 
dictionary8 = {'REFERRAL_TO_TREATMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcReferralTreatment', 'ASSESSMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcAssessment', 
               'TREATMENT_PLANNING_NUMBER_OF_SESSIONS_RECEIVED':'SvcTreatmentPlanning', 
               'INDIVIDUAL_COUNSELING_NUMBER_OF_SESSIONS_RECEIVED':'SvcIndividualCouns'}

#Renaming 
df.rename(columns = dictionary8,inplace = True)

#Creating variable to replace null values
col_null14 = ['SvcReferralTreatment', 'SvcTreatmentPlanning', 'SvcIndividualCouns', 'SvcAssessment']

#Replacing null values with 0
df[col_null14] = df[col_null14].fillna(0)

# filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcReferralTreatment']
    else:
        pass
df['SvcReferralTreatment'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcTreatmentPlanning']
    else:
        pass
df['SvcTreatmentPlanning'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcIndividualCouns']
    else:
        pass
df['SvcIndividualCouns'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcAssessment']
    else:
        pass
df['SvcAssessment'] = df.apply(interview_type, axis = 1)


In [97]:
#Creating columns with missing values 

# filtering based on interview type
df['SvcGroupCouns'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcGroupCouns']
    else:
        pass
df['SvcGroupCouns'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcContingencyManagement'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcContingencyManagement']
    else:
        pass
df['SvcContingencyManagement'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcCommunityReinforcement'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcCommunityReinforcement']
    else:
        pass
df['SvcCommunityReinforcement'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcCBT'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcCBT']
    else:
        pass
df['SvcCBT'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcFamilyMarriageCouns'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcFamilyMarriageCouns']
    else:
        pass
df['SvcFamilyMarriageCouns'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcCoOccurring'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcCoOccurring']
    else:
        pass
df['SvcCoOccurring'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcPharmacological'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcPharmacological']
    else:
        pass
df['SvcPharmacological'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcHIVAIDSCouns'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcHIVAIDSCouns']
    else:
        pass
df['SvcHIVAIDSCouns'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcCulturalInterventions'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcCulturalInterventions']
    else:
        pass
df['SvcCulturalInterventions'] = df.apply(interview_type, axis = 1)

# filtering based on interview type
df['SvcOtherClinicalCouns'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOtherClinicalCouns']
    else:
        pass
df['SvcOtherClinicalCouns'] = df.apply(interview_type, axis = 1)

df['SvcOtherClinicalCounsSpec'] = " "

# filtering based on interview type
df['SvcChildCare'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcChildCare']
    else:
        pass
df['SvcChildCare'] = df.apply(interview_type, axis = 1)

In [98]:
#Creating dictionary to rename
dictionary11 = {'CASE_MANAGEMENT_NUMBER_OF_DAYS_RECEIVED':'SvcCaseManagement', 
                'INTENSIVE_OUTPATIENT_TREATMENT_NUMBER_OF_DAYS_RECEIVED':'SvcIntensiveOutpatient', 
                'OUTREACH_NUMBER_OF_DAYS_RECEIVED':'SvcOutreach',
                'RECOVERY_SUPPORT_NUMBER_OF_DAYS_RECEIVED':'SvcRecoverySupport',
               'SCREENING_NUMBER_OF_SESSIONS_RECEIVED':'SvcScreening',
                'BRIEF_INTERVENTION_NUMBER_OF_SESSIONS_RECEIVED':'SvcBriefIntervention',
               'FAMILY_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcFamilyServices', 
                'PRE_EMPLOYMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcPreEmployment'}

#Renaming columns 
df.rename(columns = dictionary11,inplace = True)

#Replacing null values with 0
col_fill0 =['SvcCaseManagement', 'SvcIntensiveOutpatient', 'SvcOutreach','SvcRecoverySupport',
            'SvcScreening','SvcBriefIntervention','SvcFamilyServices', 'SvcPreEmployment']

df[col_fill0] = df[col_fill0].fillna(0)

#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcCaseManagement']
    else:
        pass
df['SvcCaseManagement'] = df.apply(interview_type, axis = 1)


#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcIntensiveOutpatient']
    else:
        pass
df['SvcIntensiveOutpatient'] = df.apply(interview_type, axis = 1)


#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOutreach']
    else:
        pass
df['SvcOutreach'] = df.apply(interview_type, axis = 1)


#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcRecoverySupport']
    else:
        pass
df['SvcRecoverySupport'] = df.apply(interview_type, axis = 1)


#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcScreening']
    else:
        pass
df['SvcScreening'] = df.apply(interview_type, axis = 1)

#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcBriefIntervention']
    else:
        pass
df['SvcBriefIntervention'] = df.apply(interview_type, axis = 1)

#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcFamilyServices']
    else:
        pass
df['SvcFamilyServices'] = df.apply(interview_type, axis = 1)


#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcPreEmployment']
    else:
        pass
df['SvcPreEmployment'] = df.apply(interview_type, axis = 1)

In [99]:
#Renaming columns 
df.rename(columns = {'EMPLOYMENT_COACHING_NUMBER_OF_SESSIONS_RECEIVED':'SvcEmploymentCoaching', 
                     'INDIVIDUAL_SERVICES_COORDINATION_NUMBER_OF_SESSIONS_RECEIVED':'SvcIndividualCoord', 
                     'TRANSPORTATION_NUMBER_OF_SESSIONS_RECEIVED': 'SvcTransportation'}, inplace = True)

#Creating variable for columns
col1 = ['SvcEmploymentCoaching','SvcIndividualCoord','SvcTransportation']

#Replacing null values
df[col1] = df[col1].fillna(0)


#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcEmploymentCoaching']
    else:
        pass
df['SvcEmploymentCoaching'] = df.apply(interview_type, axis = 1)

#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcIndividualCoord']
    else:
        pass
df['SvcIndividualCoord'] = df.apply(interview_type, axis = 1)

#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcTransportation']
    else:
        pass
df['SvcTransportation'] = df.apply(interview_type, axis = 1)

In [100]:
#Creating column with missing values

df['SvcHIVAIDSServices'] = 0
def interview_type(row):
    if row['InterviewType'] == 5: # filtering based on interview type
        return row['SvcHIVAIDSServices']
    else:
        pass
df['SvcHIVAIDSServices'] = df.apply(interview_type, axis = 1)


df.rename(columns = {'TRANSITIONAL_DRUG_FREE_HOUSING_NUMBER_OF_SESSIONS_RECEIVED':'SvcDrugFreeHousing',
                     'HOUSING_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcHousingSupport',
                     'HEALTH_INSURANCE_ENROLLMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcHealthInsuranceEnrollment',
                     'OTHER_CASE_MANAGEMENT_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherCaseMgmt',
                     'OTHER_CASE_MANAGEMENT_SERVICES_RECEIVED': 'SvcOtherCaseMgmtSpec'}, inplace = True)

#Creating variable 'col' to replace missing values
col2 = ['SvcDrugFreeHousing','SvcHousingSupport','SvcHealthInsuranceEnrollment','SvcOtherCaseMgmt']

#Replacing missing values with 0
df[col2] = df[col2].fillna(0)

#Editing string values
# df['SvcOtherCaseMgmtSpec'] = df['SvcOtherCaseMgmtSpec'].str.title()


#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcDrugFreeHousing']
    else:
        pass
df['SvcDrugFreeHousing'] = df.apply(interview_type, axis = 1)

#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcHousingSupport']
    else:
        pass
df['SvcHousingSupport'] = df.apply(interview_type, axis = 1)

#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcHealthInsuranceEnrollment']
    else:
        pass
df['SvcHealthInsuranceEnrollment'] = df.apply(interview_type, axis = 1)

#filtering based on interview type
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOtherCaseMgmt']
    else:
        pass
df['SvcOtherCaseMgmt'] = df.apply(interview_type, axis = 1)



In [101]:
#Renaming columns 
df.rename(columns = {'MEDICAL_CARE_NUMBER_OF_SESSIONS_RECEIVED':'SvcMedicalCare'}, inplace = True)

#Replacing missing values with 0
df['SvcMedicalCare'] = df['SvcMedicalCare'].fillna(0) 
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcMedicalCare'] #filtering based on interview type
    else:
        pass
df['SvcMedicalCare'] = df.apply(interview_type, axis = 1)

#Creating columns with missing values 
df['SvcAlcoholDrugTesting'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcAlcoholDrugTesting'] #filtering based on interview type
    else:
        pass
df['SvcAlcoholDrugTesting'] = df.apply(interview_type, axis = 1)

df['SvcOBGYN'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOBGYN'] #filtering based on interview type
    else:
        pass
df['SvcOBGYN'] = df.apply(interview_type, axis = 1)


df['SvcHIVAIDSMedical'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcHIVAIDSMedical'] #filtering based on interview type
    else:
        pass
df['SvcHIVAIDSMedical'] = df.apply(interview_type, axis = 1)

df['SvcHepatitisSupport'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcHepatitisSupport'] #filtering based on interview type
    else:
        pass
df['SvcHepatitisSupport'] = df.apply(interview_type, axis = 1)


df['SvcOtherSTISupport'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOtherSTISupport'] #filtering based on interview type
    else:
        pass
df['SvcOtherSTISupport'] = df.apply(interview_type, axis = 1)

In [102]:
#Renaming columns
df.rename(columns = {'DENTAL_CARE_NUMBER_OF_SESSIONS_RECEIVED':'SvcDental', 
                     'OTHER_MEDICAL_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherMedical',
                     'OTHER_MEDICAL_SERVICES_RECEIVED':'SvcOtherMedicalSpec',
                     'CONTINUING_CARE_NUMBER_OF_SESSIONS_RECEIVED':'SvcContinuingCare',
                     'RELAPSE_PREVENTION_NUMBER_OF_SESSIONS_RECEIVED':'SvcRelapsePrevention'}, inplace = True)

#Creating variable col1 to replace missing values with 0
col3 = ['SvcDental','SvcOtherMedical','SvcContinuingCare','SvcRelapsePrevention']

#Replacing missing values with 0
df[col3] = df[col3].fillna(0) 

#Data Cleaning
# df['SvcOtherMedicalSpec'] = df['SvcOtherMedicalSpec'].str.title() 

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcDental'] #filtering based on interview type
    else:
        pass
df['SvcDental'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOtherMedical'] #filtering based on interview type
    else:
        pass
df['SvcOtherMedical'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcContinuingCare'] #filtering based on interview type
    else:
        pass
df['SvcContinuingCare'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcRelapsePrevention'] #filtering based on interview type
    else:
        pass
df['SvcRelapsePrevention'] = df.apply(interview_type, axis = 1)

In [103]:
#Renaming columns
df.rename(columns = {'RECOVERY_COACHING_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryCoaching','SELF_HELP_MUTUAL_SUPPORT_GROUP_NUMBER_OF_SESSIONS_RECEIVED':'SvcSelfHelpSupport','SPIRITUAL_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcSpiritualSupport','OTHER_AFTER_CARE_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherAfterCare','OTHER_AFTER_CARE_SERVICES_RECEIVED':'SvcOtherAfterCareSpec'}, inplace = True)

#Creating variable col2 to replace missing values
col4 = ['SvcRecoveryCoaching','SvcSelfHelpSupport','SvcSpiritualSupport','SvcOtherAfterCare']

#Replacing missing values with 0
df[col4] = df[col4].fillna(0)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcRecoveryCoaching'] #filtering based on interview type
    else:
        pass
df['SvcRecoveryCoaching'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcSelfHelpSupport'] #filtering based on interview type
    else:
        pass
df['SvcSelfHelpSupport'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcSpiritualSupport'] #filtering based on interview type
    else:
        pass
df['SvcSpiritualSupport'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOtherAfterCare'] #filtering based on interview type
    else:
        pass
df['SvcOtherAfterCare'] = df.apply(interview_type, axis = 1)

In [104]:
#Renaming column
df.rename(columns = {'SUBSTANCE_MISUSE_EDUCATION_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcSubstanceAbuseEdu'}, inplace = True )

#Replacing missing values with 0
df['SvcSubstanceAbuseEdu'] = df['SvcSubstanceAbuseEdu'].fillna(0) 
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcSubstanceAbuseEdu'] #filtering based on interview type
    else:
        pass
df['SvcSubstanceAbuseEdu'] = df.apply(interview_type, axis = 1)

df['SvcHIVAIDSEdu'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcHIVAIDSEdu'] #filtering based on interview type
    else:
        pass
df['SvcHIVAIDSEdu'] = df.apply(interview_type, axis = 1)

df['SvcHepatitisEdu'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcHepatitisEdu'] #filtering based on interview type
    else:
        pass
df['SvcHepatitisEdu'] = df.apply(interview_type, axis = 1)

df['SvcOtherSTIEdu'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOtherSTIEdu'] #filtering based on interview type
    else:
        pass
df['SvcOtherSTIEdu'] = df.apply(interview_type, axis = 1)

df['SvcNaloxoneTraining'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcNaloxoneTraining'] #filtering based on interview type
    else:
        pass
df['SvcNaloxoneTraining'] = df.apply(interview_type, axis = 1)

df['SvcFentanylTestStripTraining'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcFentanylTestStripTraining'] #filtering based on interview type
    else:
        pass
df['SvcFentanylTestStripTraining'] = df.apply(interview_type, axis = 1)

df['SvcOtherEdu'] = 0
def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOtherEdu'] #filtering based on interview type
    else:
        pass
df['SvcOtherEdu'] = df.apply(interview_type, axis = 1)


df['SvcOtherEduSpec'] = " "

In [105]:
#Creating copy of RECOVERY_HOUSING_NUMBER_OF_SESSIONS_RECEIVED column
df['RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED_1'] = df['RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED']

#Renaming columns 
df.rename(columns = {'PEER_COACHING_OR_MENTORING_NUMBER_OF_SESSIONS_RECEIVED':'SvcPeerCoaching', 
                     'VOCATIONAL_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcVocational',
                     'RECOVERY_HOUSING_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryHousing',
                     'RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryPlanningRSS',
                     'RECOVERY_SUPPORT_SPECIFIC_CASE_MANAGEMENT_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryCaseManagement',
                     'ALCOHOL_AND_DRUG_FREE_SOCIAL_ACTIVITIES_NUMBER_OF_SESSIONS_RECEIVED':'SvcDrugFreeSocial'}, inplace = True)

#Creating variable col3
col5 = ['SvcPeerCoaching','SvcVocational','SvcRecoveryHousing','SvcRecoveryCaseManagement','SvcDrugFreeSocial'] 

#Replacing missing values with 0
df[col5] = df[col5].fillna(0)

#inputting values in the 'SvcRecoveryPlanningRSS' column
df['SvcRecoveryPlanningRSS'] = df['SvcRecoveryPlanningRSS'].fillna(0)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcPeerCoaching'] #filtering based on interview type
    else:
        pass
df['SvcPeerCoaching'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcVocational'] #filtering based on interview type
    else:
        pass
df['SvcVocational'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcRecoveryHousing'] #filtering based on interview type
    else:
        pass
df['SvcRecoveryHousing'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcRecoveryCaseManagement'] #filtering based on interview type
    else:
        pass
df['SvcRecoveryCaseManagement'] = df.apply(interview_type, axis = 1)


def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcDrugFreeSocial'] #filtering based on interview type
    else:
        pass
df['SvcDrugFreeSocial'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcRecoveryPlanningRSS'] #filtering based on interview type
    else:
        pass
df['SvcRecoveryPlanningRSS'] = df.apply(interview_type, axis = 1)


In [106]:
#Renaming columns
df.rename(columns = {'INFORMATION_AND_REFERRAL_NUMBER_OF_SESSIONS_RECEIVED':'SvcInformationReferral','OTHER_RECOVERY_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherRecoverySupport','OTHER_RECOVERY_SUPPORT_SERVICES_RECEIVED':'SvcOtherRecoverySupportSpec','OTHER_PEER_TO_PEER_RECOVERY_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherPeerRecovery','OTHER_PEER_TO_PEER_RECOVERY_SUPPORT_RECEIVED':'SvcOtherPeerRecoverySpec'}, inplace = True)

#Creating variable col4 to replace missing values
col6 = ['SvcInformationReferral','SvcOtherRecoverySupport','SvcOtherPeerRecovery']

#Replacing missing values with 0
df[col6] = df[col6].fillna(0)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcInformationReferral'] #filtering based on interview type
    else:
        pass
df['SvcInformationReferral'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOtherRecoverySupport'] #filtering based on interview type
    else:
        pass
df['SvcOtherRecoverySupport'] = df.apply(interview_type, axis = 1)

def interview_type(row):
    if row['InterviewType'] == 5:
        return row['SvcOtherPeerRecovery'] #filtering based on interview type
    else:
        pass
df['SvcOtherPeerRecovery'] = df.apply(interview_type, axis = 1)


In [107]:
#Replacing null values with -9
df['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'] = df['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'].fillna(-9)

#Creating function to create column AttendedMostPlannedServices
def AttendedMost_PlannedServices (row):
    if row['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'] == 'Yes':
        return 1
    elif row['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'] == 'No':
        return 0
    elif row['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'] == -9:
        return -9
    
    
def interview_type(row):
    if row['InterviewType'] == 5:#filtering based on interview type
        return AttendedMost_PlannedServices (row) 
    else:
        pass
    
#Creating column AttendedMostPlannedServices via applying the function AttendedMost_PlannedServices (x)
df['AttendedMostPlannedServices'] = df.apply(interview_type, axis = 1)

In [108]:
#Replacing null values with -9
df['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'] = df['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'].fillna(-9)

#Creating function to create column ReceivedTelehealthServices
def Received_Telehealth_Services (row):
    if row['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'] == 'Yes':
        return 1
    elif row['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'] == 'No':
        return 0
    elif row['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'] == -9:
        return -9

def interview_type(row):
    if row['InterviewType'] == 5:#filtering based on interview type
        return  Received_Telehealth_Services (row)
    else:
        pass
#Creating column ReceivedTelehealthServices via applying the function Received_Telehealth_Services (x)
df['ReceivedTelehealthServices'] = df.apply(interview_type, axis = 1)

In [109]:
#Replacing null values with -9
df['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] = df['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'].fillna(-9)

#Creating a function to create column PreviouslyDiagnosedOUD
def Previously_DiagnosedOUD (row):
    if row['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] == 'Yes':
        return 1
    elif row['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] == 'No [SKIP TO QUESTION 5]':
        return 0
    elif row['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] == -9:
        return -9
    
def interview_type(row):
    if row['InterviewType'] == 5:#filtering based on interview type
        return  Previously_DiagnosedOUD (row)
    else:
        pass
    
#Creating column PreviouslyDiagnosedOUD via applying the function Previously_DiagnosedOUD (x)
df['PreviouslyDiagnosedOUD'] = df.apply(interview_type, axis = 1)

In [110]:
#Replacing null values with -9
df['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] = df['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]
    
#Creating function to create column kOUDMedMethadone
def kOUD_Med_Methadone (row):
    if row ['PreviouslyDiagnosedOUD']== -9:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Methadone':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5:#filtering based on interview type
        return  kOUD_Med_Methadone (row)
    else:
        pass    
#Creating column kOUDMedMethadone via applying the function kOUD_Med_Methadone (x)
df['kOUDMedMethadone'] = df.apply(interview_type, axis = 1)

In [111]:
# creating column kOUDMedMethadoneDays
df.rename(columns = {'METHADONE_NUMBER_OF_DAYS':'kOUDMedMethadoneDays'}, inplace = True)

def kOUDMedMethadoneDays (row):
    if row['kOUDMedMethadone'] == 1 :
        return row['kOUDMedMethadoneDays']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kOUDMedMethadoneDays (row)
    else:
        pass      
df['kOUDMedMethadoneDays'] = df.apply(interview_type, axis = 1)

# data cleaning
# df['kOUDMedMethadoneDays'] = df['kOUDMedMethadoneDays'].apply(lambda x: thirty(x))

In [112]:
#Creating function to create column kOUDMedBuprenorphine
def kOUDMed_Buprenorphine (row):
    if row['PreviouslyDiagnosedOUD'] == -9:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Buprenorphine':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kOUDMed_Buprenorphine (row)
    else:
        pass      
                                       
#Creating column kOUDMedMethadone via applying the function kOUDMed_Buprenorphine (x)
df['kOUDMedBuprenorphine'] = df.apply(interview_type, axis = 1)

In [113]:
# creating column kOUDMedBuprenorphineDays
df.rename(columns = {'BUPRENORPHINE_NUMBER_OF_DAYS':'kOUDMedBuprenorphineDays'}, inplace = True)

def kOUDMedBuprenorphineDays (row):
    if row['kOUDMedBuprenorphine'] == 1:
        return row['kOUDMedBuprenorphineDays']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kOUDMedBuprenorphineDays (row)
    else:
        pass      
                 

df['kOUDMedBuprenorphineDays'] = df.apply(interview_type, axis = 1)

# data cleaning
df['kOUDMedBuprenorphineDays'] = df['kOUDMedBuprenorphineDays'].apply(lambda x : thirty(x))


In [114]:
#Creating function to create column kOUDMedNaltrexone
def kOUDMed_Naltrexone (row):
    if row['PreviouslyDiagnosedOUD']== -9:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Naltrexone':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kOUDMed_Naltrexone (row)
    else:
        pass  
    
#Creating column kOUDMedNaltrexone via applying the function kOUDMed_Naltrexone (x)
df['kOUDMedNaltrexone'] = df.apply(interview_type, axis = 1)

In [115]:
# creating column kOUDMedNaltrexoneDays
def kOUDMedNaltrexoneDays (row):
    if row['kOUDMedNaltrexone'] == 1:
        return row['NALTREXONE_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kOUDMedNaltrexoneDays (row)
    else:
        pass  

df['kOUDMedNaltrexoneDays'] = df.apply(interview_type, axis = 1)

In [116]:
# creating column kOUDMedXRNaltrexone
def kOUDMedXRNaltrexone(row):
    if row['PreviouslyDiagnosedOUD']== -9:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Extended-release Naltrexone':
            return 1
    else:
            return 0
        
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kOUDMedXRNaltrexone(row)
    else:
        pass

df['kOUDMedXRNaltrexone'] = df.apply(interview_type, axis = 1)

In [117]:
# creating column kOUDMedXRNaltrexoneDoses
def kOUDMedXRNaltrexoneDoses(row):
    if row['kOUDMedXRNaltrexone'] == 1:
        return row['NUMBER_OF_DAYS_RECEIVED_EXTENDED_RELEASE_NALTREXONE']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kOUDMedXRNaltrexoneDoses(row)
    else:
        pass
    
df['kOUDMedXRNaltrexoneDoses'] = df.apply(interview_type, axis = 1)

In [118]:
# creating the column kOUDMedNotReceived
def kOUDMedNotReceived(row):
    if row['PreviouslyDiagnosedOUD']== -9:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive an FDA-approved medication for a diagnosed opioid use disorder [SKIP TO QUESTION 5]':
        return 1
    else:
        return 0
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kOUDMedNotReceived(row)
    else:
        pass

df['kOUDMedNotReceived'] = df.apply(interview_type, axis = 1)

In [119]:
# creating column OUDMedTakenAsPrescribed
def OUDMedTakenAsPrescribed (row):
    if row['PreviouslyDiagnosedOUD']== -9:
        return -1
    elif row['kOUDMedMethadone'] == 1:
        return 1
    elif row['kOUDMedBuprenorphine'] == 1:
        return 1
    elif row['kOUDMedNaltrexone'] == 1:
        return 1
    elif row['kOUDMedXRNaltrexone'] == 1:
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  OUDMedTakenAsPrescribed (row)
    else:
        pass
    
df['OUDMedTakenAsPrescribed'] = df.apply(interview_type, axis = 1)

In [120]:
df['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'] = df['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'].fillna(-9)
# creating column PreviouslyDiagnosedAUD
def PreviouslyDiagnosedAUD (row):
    if row['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'] == 'Yes':
        return 1
    elif row['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'] == 'No':
        return 0
    else:
        return -9
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  PreviouslyDiagnosedAUD (row)
    else:
        pass    

df['PreviouslyDiagnosedAUD'] = df.apply(interview_type, axis = 1)

In [121]:
df['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] = df['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]

def kAUDMedNaltrexone (row):
    if row['PreviouslyDiagnosedAUD'] == -9:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Naltrexone':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kAUDMedNaltrexone (row)
    else:
        pass 
    
df['kAUDMedNaltrexone'] = df.apply(interview_type, axis = 1)

In [122]:
def kAUDMedNaltrexoneDays (row):
    if row['kAUDMedNaltrexone'] == 1:
        return row['NALTREXONE_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kAUDMedNaltrexoneDays (row)
    else:
        pass 
    
df['kAUDMedNaltrexoneDays'] = df.apply(interview_type, axis = 1)

In [123]:
def kAUDMedXRNaltrexone (row):
    if row['PreviouslyDiagnosedAUD'] == -9:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Extended-release naltrexone':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kAUDMedXRNaltrexone (row)
    else:
        pass     

    
df['kAUDMedXRNaltrexone'] = df.apply(interview_type, axis = 1)

In [124]:
def kAUDMedXRNaltrexoneDoses (row):
    if row['kAUDMedXRNaltrexone'] == 1:
        return row['EXTENDED_RELEASE_NALTREXONE_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kAUDMedXRNaltrexoneDoses (row)
    else:
        pass     

df['kAUDMedXRNaltrexoneDoses'] = df.apply(interview_type, axis = 1)

# df['kAUDMedXRNaltrexoneDoses'] = df['kAUDMedXRNaltrexone'].apply(lambda x: doses (x))

In [125]:
def kAUDMedDisulfiram (row):
    if row['PreviouslyDiagnosedAUD'] == -9:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Disulfiram':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kAUDMedDisulfiram (row)
    else:
        pass 
    
df['kAUDMedDisulfiram'] =  df.apply(interview_type, axis = 1)

In [126]:
def kAUDMedDisulfiramDays (row):
    if row['kAUDMedDisulfiram'] == 1:
        return row['NUMBER_OF_DAYS_RECEIVED_DISULFIRAM']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kAUDMedDisulfiramDays (row)
    else:
        pass     

df['kAUDMedDisulfiramDays'] = df.apply(interview_type, axis = 1)

In [127]:
def kAUDMedAcamprosate (row):
    if row['PreviouslyDiagnosedAUD'] == -9:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Acamprosate':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kAUDMedAcamprosate (row)
    else:
        pass     
    

df['kAUDMedAcamprosate'] = df.apply(interview_type, axis = 1)

In [128]:
def kAUDMedAcamprosateDays (row):
    if row['kAUDMedAcamprosate'] == 1:
        return row['NUMBER_OF_DAYS_RECEIVED_ACAMPROSATE']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kAUDMedAcamprosate (row)
    else:
        pass 
    

df['kAUDMedAcamprosateDays'] = df.apply(interview_type, axis = 1)

In [129]:
#Creating function to create column kAUDMedNotReceived
def  kAUDMed_NotReceived (row):
    if row['PreviouslyDiagnosedAUD'] == -9:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive an FDA-approved medication for an alcohol use disorder [SKIP TO QUESTION 6]':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kAUDMed_NotReceived (row)
    else:
        pass     

#Creating column kAUDMedNotReceived via applying the function kAUDMed_NotReceived (x)
df['kAUDMedNotReceived'] = df.apply(interview_type, axis = 1)

In [130]:
#Creating column with missing values
def AUDMedTakenAsPrescribed (row):
    if row['PreviouslyDiagnosedAUD'] == -9:
        return -1
    elif row['kAUDMedNaltrexone'] == 1:
        return 1
    elif row['kAUDMedXRNaltrexone'] == 1:
        return 1
    elif row['kAUDMedDisulfiram'] == 1:
        return 1
    elif row['kAUDMedAcamprosate'] == 1:
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return AUDMedTakenAsPrescribed (row)
    else:
        pass  
    
df['AUDMedTakenAsPrescribed'] = df.apply(interview_type, axis = 1)

In [131]:
#Replacing null values with -9
df['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] = df['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'].fillna(-9)

#Creating function to create column PreviouslyDiagnosedStUD
def Previously_DiagnosedStUD (row):
    if row['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] == 'Yes':
        return 1
    elif row['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] == 'No [SKIP TO QUESTION 7]':
        return 0
    elif row['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] == -9:
        return -9
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return Previously_DiagnosedStUD (row)
    else:
        pass      

#Creating column PreviouslyDiagnosedStUD via applying the function Previously_DiagnosedStUD (x)
df['PreviouslyDiagnosedStUD'] = df.apply(interview_type, axis = 1)

In [132]:
#Replacing null valules with -9
df['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] = df['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]

#Creating function to create column kStUDIntContMgmt
def kStUDIntContMgmt (row):
    if row['PreviouslyDiagnosedStUD'] == -9:
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Contingency Management':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kStUDIntContMgmt (row)
    else:
        pass   

#Creating column kStUDIntCommReinf via applying the function kStUDInt_CommReinf (x)
df['kStUDIntContMgmt'] = df.apply(interview_type, axis = 1)

In [133]:
def kStUDIntContMgmtDays (row):
    if row['kStUDIntContMgmt'] == 1:
        return row['CONTINGENCY_MANAGEMENT_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kStUDIntContMgmtDays (row)
    else:
        pass    

df['kStUDIntContMgmtDays'] = df.apply(interview_type, axis = 1)

In [134]:
#Creating function to create column kStUDIntCommReinf
def kStUDIntCommReinf (row):
    if row['PreviouslyDiagnosedStUD'] == -9:
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Community Reinforcement':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kStUDIntCommReinf (row)
    else:
        pass 

#Creating column kStUDIntCommReinf via applying the function kkStUDIntCommReinf (x)
df['kStUDIntCommReinf'] = df.apply(interview_type, axis = 1)

In [135]:
def kStUDIntCommReinfDays (row):
    if row['kStUDIntCommReinf'] == 1:
        return row['COMMUNITY_REINFORCEMENT_NUMBER_OF_DAYS']
    else:
        pass

def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kStUDIntCommReinfDays (row)
    else:
        pass     

df['kStUDIntCommReinfDays'] = df.apply(interview_type, axis = 1)

In [136]:
def kStUDIntCBT (row):
    if row['PreviouslyDiagnosedStUD'] == -9:
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Cognitive Behavioral Therapy':
        return 1 
    else:
        return 0
    

def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kStUDIntCBT (row)
    else:
        pass     

df['kStUDIntCBT'] = df.apply(interview_type, axis = 1)

In [137]:
def kStUDIntCBTDays (row):
    if row['kStUDIntCBT'] == 1 :
        return row['COGNITIVE_BEHAVIORAL_THERAPY_NUMBER_OF_DAYS'] 
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kStUDIntCBTDays (row)
    else:
        pass    

df['kStUDIntCBTDays'] = df.apply(interview_type, axis = 1)

In [138]:
#Creating function to create column kStUDIntOther
def kStUDIntOther (row):
    if row['PreviouslyDiagnosedStUD'] == -9:
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Other treatment approach':
        return 1
    else:
        return 0

def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kStUDIntOther (row)
    else:
        pass      
    
#Creating column kStUDIntOther via applying the functionkStUD_IntOther (x)
df['kStUDIntOther'] = df.apply(interview_type, axis = 1)

In [139]:
def kStUDIntOtherDays (row):
    if row['kStUDIntOther'] == 1:
        return row['OTHER_TREATMENT_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kStUDIntOtherDays (row)
    else:
        pass 
    
    
df['kStUDIntOtherDays'] = df.apply(interview_type, axis = 1)

In [140]:
#Creating function to create column kStUDIntNotReceived
def kStUDIntNotReceived (row):
    if row['PreviouslyDiagnosedStUD'] == -9:
        return -1 
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive any intervention for a stimulant use disorder [SKIP TO QUESTION 7]':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  kStUDIntNotReceived (row)
    else:
        pass 

#Creating column kStUDIntNotReceived via applying the function kStUD_IntNotReceived (x)
df['kStUDIntNotReceived'] = df.apply(interview_type, axis = 1)

In [141]:
def StUDIntAttended (row):
    if row['PreviouslyDiagnosedStUD'] == -9 :
        return -1
    elif row['kStUDIntContMgmt'] == 1:
        return 1
    elif row['kStUDIntCommReinf'] == 1:
        return 1
    elif row['kStUDIntCBT'] == 1:
        return 1
    elif row['kStUDIntOther'] == 1:
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  StUDIntAttended (row)
    else:
        pass     

    
df['StUDIntAttended'] = df.apply(interview_type, axis = 1)

In [142]:
#Replacing null values with -9
df['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] = df['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'].fillna(-9)

#Creating function to create column PreviouslyDiagnosedTUD
def PreviouslyDiagnosedTUD (row):
    if row['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] == 'Yes':
        return 1
    elif row['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] == 'No [THE DISCHARGE INTERVIEW IS COMPLETE.]':
        return 0
    elif row['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] == -9:
        return -9
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return  PreviouslyDiagnosedTUD (row)
    else:
        pass     

#Creating column PreviouslyDiagnosedTUD via applying the function 
df['PreviouslyDiagnosedTUD'] = df.apply(interview_type, axis = 1)

In [143]:
df['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] = df['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]

def kTUDMedNicotineRepl (row):
    if row['PreviouslyDiagnosedTUD'] == -9:
        return -1
    elif row['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Nicotine Replacement':
        return 1
    else:
        return 0  
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kTUDMedNicotineRepl (row)
    else:
        pass
    
df['kTUDMedNicotineRepl'] = df.apply(interview_type, axis = 1)

In [144]:
def kTUDMedNicotineReplDays (row):
    if row['kTUDMedNicotineRepl'] == 1:
        return row['NICOTINE_REPLACEMENT_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kTUDMedNicotineReplDays (row)
    else:
        pass
    
df['kTUDMedNicotineReplDays'] = df.apply(interview_type, axis = 1)

In [145]:
def kTUDMedBupropion (row):
    if row['PreviouslyDiagnosedTUD'] == -9:
        return -1
    elif row['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Bupropion':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kTUDMedBupropion (row)
    else:
        pass    

df['kTUDMedBupropion'] = df.apply(interview_type, axis = 1)

In [146]:
def kTUDMedBupropionDays (row):
    if row['kTUDMedBupropion'] == 1:
        return row['BUPROPION_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kTUDMedBupropionDays (row)
    else:
        pass  
    
df['kTUDMedBupropionDays'] = df.apply(interview_type, axis = 1)

In [147]:
def kTUDMedVarenicline (row):
    if row['PreviouslyDiagnosedTUD'] == -9:
        return -1
    elif row ['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Varenicline':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kTUDMedVarenicline (row)
    else:
        pass  
    
df['kTUDMedVarenicline'] = df.apply(interview_type, axis = 1)

In [148]:
def kTUDMedVareniclineDays (row):
    if row['kTUDMedVarenicline'] == 1:
        return row['VARENICLINE_NUMBER_OF_DAYS']
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kTUDMedVareniclineDays (row)
    else:
        pass  
    
df['kTUDMedVareniclineDays'] = df.apply(interview_type, axis = 1)

In [149]:
def kTUDMedNotReceived (row):
    if row['PreviouslyDiagnosedTUD'] == -9:
        return -1
    elif row['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive an FDA-approved medication for a tobacco use disorder [THE DISCHARGE INTERVIEW IS COMPLETE.]':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return kTUDMedNotReceived (row)
    else:
        pass
    
df['kTUDMedNotReceived'] = df.apply(interview_type, axis = 1)

In [150]:
def TUDMedTakenAsPrescribed (row):
    if row['PreviouslyDiagnosedTUD'] == -9:
        return -1
    elif row['kTUDMedNicotineRepl'] == 1:
        return 1
    elif row['kTUDMedBupropion'] == 1:
        return 1
    elif row['kTUDMedVarenicline'] == 1:
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return TUDMedTakenAsPrescribed (row)
    else:
        pass    

df['TUDMedTakenAsPrescribed'] = df.apply(interview_type, axis = 1)

In [151]:
# creating datetime object
now = dt.now().strftime('%m/%d/%Y')

#Creating the IntakeDate and InterviewCreateDate columns 
df['InterviewCreateDate'] = now

In [152]:
#Renaming column to create column SvcRecoveryPlanning
df.rename(columns = {'RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED_1':'SvcRecoveryPlanning'},inplace = True)

#Replacing null values with 0
df['SvcRecoveryPlanning'] = df['SvcRecoveryPlanning'].fillna(0)
def interview_type(row):
    if row['InterviewType'] == 5: #filtering based on interview type
        return row['SvcRecoveryPlanning']
    else:
        pass    

df['SvcRecoveryPlanning'] = df.apply(interview_type, axis = 1)

In [153]:
#Inserting blank columns from section H

section_h_blank_col = [
'ReunitedChild',
'ReunitedChildWithAgcySupe',
'ReunitedChildWithoutAgcySupe',
'AvoidedOutOfHomePlacement',
'H1NoneOfAbove',
'HelpObPrivateHealthInsurance',
'HelpObMedicaid',
'HelpObMedicare',
'HelpObSSIOrSSDI',
'HelpObTANF',
'HelpObSNAP',
'HelpObOtherBenefit',
'HelpObOtherBenefitSpec',
'HelpObNoneOfTheAbove',
'EnrolledInSchool',
'EnrolledInSchoolHelp',
'EnrolledInVocTraining',
'EnrolledInVocTrainingHelp',
'CurrentlyEmployed',
'CurrentlyEmployedHelp',
'LivingInStableHousing',
'LivingInStableHousingHelp',
'MaintainFamilyResponsibilities',
'SupportToParentingAndRecovery',
'EnableFocusOnTreatment',
'SupportToParentingAndRecoveryH5',
'SexActivity',
'SexContacts',
'SexUnprot',
'SexUnprotectedHIV',
'SexUnprotectedInjDrugUser',
'SexUnprotectedHigh',
'PrEPOrTreat',
'HIVTestH',
'HIVTestFirstTime',
'HIVTestLocation',
'HIVTestReferralLocation',
'HIVTestReferralLocationSpec',
'HIVTestResultH',
'HIVTestConfirmatory',
'HIVTestConfirmatoryRes',
'TreatmentForHIVH',
'TreatmentForHIVLocation',
'TreatmentForHIVLocationSpec',
'HIVTestingPartners',
'HIVTestingPartnersNumber',
'HIVReferredPreEP',
'HIVReferredPEP',
'HIVReferredCounseling',
'HIVReferredNoMedications',
'HIVReferredNoCounseling',
'HCVTest',
'HCVTestConfirmatory',
'HCVTestResult',
'TreatmentForHCV',
'HBVTest',
'HBVTestResult',
'TreatmentForHBV',
'HepABVacc',
'HepABVaccReferral',
'PeerSupport',
'EnrolledInSchoolH8',
'EnrolledInSchoolHelpH8',
'EnrolledInVocTrainingH8',
'EnrolledInVocTrainingHelpH8',
'CurrentlyEmployedH8',
'CurrentlyEmployedHelpH8',
'LivingInStableHousingH8',
'LivingInStableHousingHelpH8',
'QualityOfLife',
'CommunicateWithProvider',
'ReduceSubstanceUse',
'ManageMentalHealthSymptoms',
'SupportRecovery',
'MentalHealthDisorderScreen',
'MentalHealthServiceReferred',
'MentalHealthServiceReceived',
'CoOccurringDisorderScreen',
'CoOccurringServiceReferred',
'CoOccurringServiceReceived',
'HelpedAvoidJusticeSystem'

]

df[section_h_blank_col] = df[section_h_blank_col] = ''

In [154]:
#Creating QAOnly column
df['QAOnly'] = ''

## Exporting GPRA 6

In [155]:
#Creating variable to export GPRA 6
GPRA6_STAR = df[['ClientID','FLWPClientContacted',	'FLWPStatus',	'FLWPStatusSpec',
                 'ReceivingServices',	'DischargeDate',	'DischargeStatusCompl',
                 'DischargeStatusTermReason',	'OtherDischargeStatTermRsnSpec',
                 'jHIVTest',	'jHIVTestResult',	'ODPreventionToolsProvided',
                 'VaccinatedStatusCOVID19',	'SvcCaseManagement',	'SvcIntensiveOutpatient',
                 'SvcInpatient',	'SvcOutpatient',	'SvcOutreach',	'SvcMethadone',
                 'SvcBuprenorphine',	'SvcNaltrexoneShortActing',	'SvcNaltrexoneLongActing',
                 'SvcDisulfiram',	'SvcAcamprosate',	'SvcNicotineReplacement',
                 'SvcBupropion',	'SvcVarenicline',	'SvcResidentialRehab',
                 'SvcHospitalInpatient',	'SvcFreeStandingRes',	'SvcAmbulatoryDetox',
                 'SvcAfterCare',	'SvcRecoverySupport',	'SvcOtherModalities',
                 'SvcOtherModalitesSpec',	'SvcScreening',	'SvcBriefIntervention',
                 'SvcBriefTreatment',	'SvcReferralTreatment',	'SvcAssessment',
                 'SvcTreatmentPlanning',	'SvcRecoveryPlanning',	'SvcIndividualCouns',
                 'SvcGroupCouns',	'SvcContingencyManagement',	'SvcCommunityReinforcement',
                 'SvcCBT',	'SvcFamilyMarriageCouns',	'SvcCoOccurring',	'SvcPharmacological',
                 'SvcHIVAIDSCouns',	'SvcCulturalInterventions',	'SvcOtherClinicalCouns',
                 'SvcOtherClinicalCounsSpec',	'SvcFamilyServices',	'SvcChildCare',	'SvcPreEmployment',
                 'SvcEmploymentCoaching',	'SvcIndividualCoord',	'SvcTransportation',	'SvcHIVAIDSServices',
                 'SvcDrugFreeHousing',	'SvcHousingSupport',	'SvcHealthInsuranceEnrollment',	'SvcOtherCaseMgmt',
                 'SvcOtherCaseMgmtSpec',	'SvcMedicalCare',	'SvcAlcoholDrugTesting',	'SvcOBGYN',	'SvcHIVAIDSMedical',
                 'SvcHepatitisSupport',	'SvcOtherSTISupport',	'SvcDental','SvcOtherMedical','SvcOtherMedicalSpec',
                 'SvcContinuingCare',	'SvcRelapsePrevention',	'SvcRecoveryCoaching',	'SvcSelfHelpSupport',
                 'SvcSpiritualSupport',	'SvcOtherAfterCare',	'SvcOtherAfterCareSpec',	'SvcSubstanceAbuseEdu',
                 'SvcHIVAIDSEdu',	'SvcHepatitisEdu',	'SvcOtherSTIEdu',	'SvcNaloxoneTraining',
                 'SvcFentanylTestStripTraining',	'SvcOtherEdu',	'SvcOtherEduSpec',	'SvcPeerCoaching',
                 'SvcVocational',	'SvcRecoveryHousing',	'SvcRecoveryPlanningRSS',	'SvcRecoveryCaseManagement',
                 'SvcDrugFreeSocial',	'SvcInformationReferral',	'SvcOtherRecoverySupport',
                 'SvcOtherRecoverySupportSpec',	'SvcOtherPeerRecovery',	'SvcOtherPeerRecoverySpec',
                 'AttendedMostPlannedServices',	'ReceivedTelehealthServices',	'PreviouslyDiagnosedOUD',
                 'kOUDMedMethadone',	'kOUDMedMethadoneDays',	'kOUDMedBuprenorphine',	'kOUDMedBuprenorphineDays',
                 'kOUDMedNaltrexone',	'kOUDMedNaltrexoneDays',	'kOUDMedXRNaltrexone','kOUDMedXRNaltrexoneDoses',
                 'kOUDMedNotReceived',	'OUDMedTakenAsPrescribed',	'PreviouslyDiagnosedAUD',	'kAUDMedNaltrexone',
                 'kAUDMedNaltrexoneDays',	'kAUDMedXRNaltrexone',	'kAUDMedXRNaltrexoneDoses',	'kAUDMedDisulfiram',
                 'kAUDMedDisulfiramDays',	'kAUDMedAcamprosate',	'kAUDMedAcamprosateDays',	'kAUDMedNotReceived',
                 'AUDMedTakenAsPrescribed',	'PreviouslyDiagnosedStUD',	'kStUDIntContMgmt',	'kStUDIntContMgmtDays',
                 'kStUDIntCommReinf',	'kStUDIntCommReinfDays',	'kStUDIntCBT',	'kStUDIntCBTDays',	'kStUDIntOther',
                 'kStUDIntOtherDays',	'kStUDIntNotReceived',	'StUDIntAttended',	'PreviouslyDiagnosedTUD',
                 'kTUDMedNicotineRepl',	'kTUDMedNicotineReplDays',	'kTUDMedBupropion',	'kTUDMedBupropionDays',
                 'kTUDMedVarenicline',	'kTUDMedVareniclineDays',	'kTUDMedNotReceived',	'TUDMedTakenAsPrescribed',
                 'ReunitedChild',	'ReunitedChildWithAgcySupe',	'ReunitedChildWithoutAgcySupe',
                 'AvoidedOutOfHomePlacement',	'H1NoneOfAbove',	'HelpObPrivateHealthInsurance',
                 'HelpObMedicaid',	'HelpObMedicare',	'HelpObSSIOrSSDI',	'HelpObTANF',	'HelpObSNAP',
                 'HelpObOtherBenefit',	'HelpObOtherBenefitSpec',	'HelpObNoneOfTheAbove',	'EnrolledInSchool',
                 'EnrolledInSchoolHelp',	'EnrolledInVocTraining',	'EnrolledInVocTrainingHelp',
                 'CurrentlyEmployed',	'CurrentlyEmployedHelp',	'LivingInStableHousing',
                 'LivingInStableHousingHelp',	'MaintainFamilyResponsibilities',	'SupportToParentingAndRecovery',
                 'EnableFocusOnTreatment',	'SupportToParentingAndRecoveryH5',	'SexActivity',	'SexContacts',
                 'SexUnprot',	'SexUnprotectedHIV',	'SexUnprotectedInjDrugUser',	'SexUnprotectedHigh',
                 'PrEPOrTreat',	'HIVTestH',	'HIVTestFirstTime',	'HIVTestLocation',	'HIVTestReferralLocation',
                 'HIVTestReferralLocationSpec',	'HIVTestResultH',	'HIVTestConfirmatory',	'HIVTestConfirmatoryRes',
                 'TreatmentForHIVH',	'TreatmentForHIVLocation',	'TreatmentForHIVLocationSpec',	'HIVTestingPartners',
                 'HIVTestingPartnersNumber',	'HIVReferredPreEP',	'HIVReferredPEP',	'HIVReferredCounseling',
                 'HIVReferredNoMedications',	'HIVReferredNoCounseling',	'HCVTest',	'HCVTestConfirmatory',
                 'HCVTestResult',	'TreatmentForHCV',	'HBVTest',	'HBVTestResult',	'TreatmentForHBV',	'HepABVacc',
                 'HepABVaccReferral',	'PeerSupport',	'EnrolledInSchoolH8',	'EnrolledInSchoolHelpH8',
                 'EnrolledInVocTrainingH8',	'EnrolledInVocTrainingHelpH8',	'CurrentlyEmployedH8',
                 'CurrentlyEmployedHelpH8',	'LivingInStableHousingH8',	'LivingInStableHousingHelpH8',
                 'QualityOfLife',	'CommunicateWithProvider',	'ReduceSubstanceUse',	'ManageMentalHealthSymptoms',
                 'SupportRecovery',	'MentalHealthDisorderScreen',	'MentalHealthServiceReferred',
                 'MentalHealthServiceReceived',	'CoOccurringDisorderScreen',	'CoOccurringServiceReferred',
                 'CoOccurringServiceReceived',	'HelpedAvoidJusticeSystem','InterviewCreateDate','QAOnly']]



#Exporting GPRA 6
GPRA6_STAR.to_csv('GPRA6_STAR.csv', index = False, header = True, encoding = 'utf-8')

